In [15]:
import sys 
sys.path.append("../../")

from br_demography  import municipality_deaths as md
import basedosdados as bd
from dotenv import load_dotenv
import os
import pandas as pd # importação de biblioteca de manipulação de dados tabulares
import matplotlib.pyplot as plt # importação de biblioteca de plotagem de gráficos 
import numpy as np # importação de biblioteca para cálculos matemáticos
from scipy.optimize import curve_fit # importação de função a ser utilizada para ajuste de modelo preditivo 
from itertools import product # importação de função voltada para otimização de laços de iteração
import warnings
from itertools import product



# Carrega as variáveis de ambiente do arquivo .env
load_dotenv(dotenv_path='../../.env')

# Use as variáveis de ambiente conforme necessário
project_id = os.getenv('GOOGLE_CLOUD_PROJECT_ID')

## Obtenção e Tratamento de Dados

### Dados de Óbitos

In [16]:
df_mun = pd.read_csv('../../br_demography/source/tab/cod_mun.csv', sep=';', index_col='mun_id')

dict_dfs = dict()

for mun_id, mun in df_mun.itertuples():
    df_deaths_mun = md.query_deaths(mun_id=mun_id, project_id=project_id, start_year=2000, end_year=2022)
    df_deaths_mun = md.standard_age_groups(df=df_deaths_mun, age_group_csv_path='../../br_demography/source/tab/faixas_etarias_censo_2000_2010.csv')
    df_deaths_mun['Município'] = mun
    df_deaths_mun = df_deaths_mun.reset_index().set_index(['Município', 'Sexo','Faixa Etária'])
    
    dict_dfs[mun] = df_deaths_mun

df_deaths = pd.concat(objs=dict_dfs.values())

Downloading: 100%|██████████| 627/627 [00:00<00:00, 2235.49rows/s]


### Dados de População por Sexo e Idade entre 2000 e 2022 

In [17]:
df_pop = pd.read_csv(filepath_or_buffer='../../br_demography/results/tab/pop_municipios_rmc_2000_2022_estimativa_intercensitaria.csv', sep=';')
#df_pop[df_pop['Faixa Etária'].isin(values=df_deaths.index)]
df_pop.set_index(keys=['Município', 'Sexo','Faixa Etária'], inplace=True)
df_pop.columns = df_pop.columns.astype(int)
df_pop.iloc[np.r_[0:2, -2:0]]

2000  2001  2002  2003  2004  2005  \
Município      Sexo      Faixa Etária                                          
Adrianópolis   Feminino  0 a 9 anos        704   681   660   639   619   599   
                         10 a 19 anos      823   798   775   752   730   708   
Doutor Ulysses Masculino 70 a 79 anos       79    79    80    81    82    83   
                         80 anos ou mais    22    23    24    26    28    30   

                                          2006  2007  2008  2009  ...  2013  \
Município      Sexo      Faixa Etária                             ...         
Adrianópolis   Feminino  0 a 9 anos        580   562   544   527  ...   486   
                         10 a 19 anos      687   667   647   628  ...   576   
Doutor Ulysses Masculino 70 a 79 anos       84    85    86    87  ...   100   
                         80 anos ou mais    31    34    36    38  ...    45   

                                          2014  2015  2016  2017  2018  2019  \
Município      Sexo      Faixa Etária                                          
Adrianópolis   Feminino  0 a 9 anos        478   470   462   455   447   440   
                         10 a 19 anos      565   555   545   534   524   515   
Doutor Ulysses Masculino 70 a 79 anos      104   108   112   117   122   127   
                         80 anos ou mais    47    49    50    52    54    56   

                                          2020  2021  2022  
Município      Sexo      Faixa Etária                       
Adrianópolis   Feminino  0 a 9 anos        433   425   419  
                         10 a 19 anos      505   496   487  
Doutor Ulysses Masculino 70 a 79 anos      132   137   143  
                         80 anos ou mais    58    60    63  

[4 rows x 23 columns]

## Cálculo de Taxa de Mortalidade

In [18]:
#mun_interest = ['Quatro Barras' ,'Doutor Ulysses', 'Adrianópolis', 'Bocaiúva do Sul', 'Rio Branco do Sul', 'Cerro Azul', 'Itaperuçu', 'Tunas do Paraná', 'Campina Grande do Sul', 'Campo Magro']

"""
mun_interest = [
    'Quatro Barras' ,'Doutor Ulysses', 'Adrianópolis', 'Bocaiúva do Sul'
    , 'Rio Branco do Sul', 'Cerro Azul', 'Itaperuçu', 'Tunas do Paraná'
    , 'Campina Grande do Sul', 'Campo Magro'
    , 'Campo do Tenente', 'Lapa', 'Quitandinha', 'Contenda', 'Agudos do Sul'
    , 'Mandirituba', 'Tijucas do Sul', 'Balsa Nova', 'Rio Negro'

    ]
"""
"""
mun_interest = [
    'Campo do Tenente', 'Lapa', 'Quitandinha', 'Contenda', 'Agudos do Sul'
    , 'Mandirituba', 'Tijucas do Sul', 'Balsa Nova', 'Rio Negro'
    

    ]

"""

'''
mun_interest = [
    'Doutor Ulysses', 'Adrianópolis', 'Cerro Azul', 'Tunas do Paraná'
    ]
'''

mun_interest = [
    'Quatro Barras', 'Bocaiúva do Sul'
    , 'Rio Branco do Sul', 'Itaperuçu'
    , 'Campina Grande do Sul', 'Campo Magro'
    , 'Campo do Tenente', 'Lapa', 'Quitandinha', 'Contenda', 'Agudos do Sul'
    , 'Mandirituba', 'Tijucas do Sul', 'Balsa Nova', 'Rio Negro'

    ]


In [19]:
df_pop = df_pop.loc[mun_interest]

In [20]:
df_deaths = df_deaths.loc[mun_interest]

In [21]:
df_pop = df_pop.reset_index().groupby(by=['Sexo', 'Faixa Etária'], as_index=False).sum().assign(Município='Restante fora Ribeira').set_index(['Município', 'Sexo', 'Faixa Etária'])

In [22]:
df_deaths = df_deaths.reset_index().groupby(by=['Sexo', 'Faixa Etária'], as_index=False).sum().assign(Município='Restante fora Ribeira').set_index(['Município', 'Sexo', 'Faixa Etária'])

In [23]:
# criação de dataframe com taxa de mortalidade a cada mil habitantes
df_mortalidade = (df_deaths / df_pop).dropna(axis=0) * 1000

df_mortalidade.drop(columns=[2020, 2021, 2022], inplace=True)


In [24]:
df_mortalidade

Ano                                                    2000        2001  \
Município             Sexo      Faixa Etária                              
Restante fora Ribeira Feminino  0 a 9 anos         1.029513    1.079500   
                                10 a 19 anos       0.423819    0.456493   
                                20 a 29 anos       0.898399    0.728833   
                                30 a 39 anos       0.717154    1.410039   
                                40 a 49 anos       3.226692    3.641902   
                                50 a 59 anos       9.222256    8.161259   
                                60 a 69 anos      22.945152   20.662413   
                                70 a 79 anos      52.969502   51.124115   
                                80 anos ou mais  118.165785  111.774744   
                      Masculino 0 a 9 anos         1.601046    1.128781   
                                10 a 19 anos       0.794334    0.955299   
                                20 a 29 anos       2.466947    2.451825   
                                30 a 39 anos       3.655279    3.687826   
                                40 a 49 anos       7.201646    6.553080   
                                50 a 59 anos      13.530135   13.551998   
                                60 a 69 anos      34.600390   26.739640   
                                70 a 79 anos      68.810289   55.365474   
                                80 anos ou mais  128.733265  125.754527   

Ano                                                    2002        2003  \
Município             Sexo      Faixa Etária                              
Restante fora Ribeira Feminino  0 a 9 anos         0.918371    0.931432   
                                10 a 19 anos       0.244285    0.485538   
                                20 a 29 anos       0.963082    0.914768   
                                30 a 39 anos       1.431672    0.998276   
                                40 a 49 anos       3.702522    3.383574   
                                50 a 59 anos      11.214777    8.490651   
                                60 a 69 anos      18.051071   18.696884   
                                70 a 79 anos      51.546392   54.335586   
                                80 anos ou mais  111.111111  108.934169   
                      Masculino 0 a 9 anos         1.416335    0.856399   
                                10 a 19 anos       1.179748    1.206273   
                                20 a 29 anos       2.968488    2.646903   
                                30 a 39 anos       3.535331    3.654189   
                                40 a 49 anos       6.415300    6.339144   
                                50 a 59 anos      13.438587   14.125056   
                                60 a 69 anos      30.658251   28.154779   
                                70 a 79 anos      53.453453   60.544612   
                                80 anos ou mais  123.902439  146.226415   

Ano                                                    2004        2005  \
Município             Sexo      Faixa Etária                              
Restante fora Ribeira Feminino  0 a 9 anos         0.653998    0.405336   
                                10 a 19 anos       0.379088    0.582052   
                                20 a 29 anos       0.867063    0.429704   
                                30 a 39 anos       1.471244    1.489334   
                                40 a 49 anos       3.023656    2.398903   
                                50 a 59 anos       8.916205    7.964161   
                                60 a 69 anos      19.141373   19.268840   
                                70 a 79 anos      41.745219   46.086509   
                                80 anos ou mais  111.694153  100.358423   
                      Masculino 0 a 9 anos         0.869807    0.742128   
                                10 a 19 anos       0.908000    1.032025   
                       

In [25]:
df_pop.sum(axis=0)

2000    281441
2001    284525
2002    287929
2003    291484
2004    295215
2005    299103
2006    303183
2007    307459
2008    311931
2009    316601
2010    321629
2011    324748
2012    328197
2013    331825
2014    335644
2015    339665
2016    343897
2017    348334
2018    353019
2019    357922
2020    363076
2021    368494
2022    374299
dtype: int64

In [26]:
from scipy.stats import iqr

# Função para o modelo exponencial decrescente
def modelo_exponencial(x, constante, coeficiente):
    return constante * np.exp(coeficiente * (-x))

# Função para o modelo Linear - casos de crescimento
def modelo_linear(x, constante, coeficiente):
    return coeficiente * x + constante

# DataFrame de resultados
resultados = pd.DataFrame(columns=['Município', 'Sexo', 'Faixa Etária','Tipo de Modelo','Constante', 'Coeficiente'])

# Combinações de sexo e faixa etária
df_mortalidade.reset_index(inplace=True)

plt.ioff()



for mun_id, mun in [('1', 'Restante fora Ribeira')]:
    print(mun)

    sexos = df_mortalidade[df_mortalidade['Município'] == mun]['Sexo'].unique()
    faixas_etarias = df_mortalidade[df_mortalidade['Município'] == mun]['Faixa Etária'].unique()

    num_plots = len(sexos) * len(faixas_etarias)
    num_colunas = 3
    num_linhas = (num_plots + num_colunas - 1) // num_colunas

    # Criar subplots para cada combinação de sexo e faixa etária
    fig, axs = plt.subplots(num_linhas, num_colunas, figsize=(15, 18))
    fig.subplots_adjust(wspace=0.5, hspace=0.5)

    for i, (sexo, faixa_etaria) in enumerate(product(sexos, faixas_etarias)):
        if i >= num_plots:
            break  # Sair se todos os subplots já estiverem preenchidos
        
        # Filtrar o DataFrame para a combinação atual
        df_filtro = df_mortalidade[(df_mortalidade['Município'] == mun) & (df_mortalidade['Sexo'] == sexo) & (df_mortalidade['Faixa Etária'] == faixa_etaria)]
        
        # Extrair os dados de anos e mortalidade
        anos = np.array(df_filtro.columns[4:], dtype=int) - 2000
        dados_mortalidade = df_filtro.iloc[0, 4:].values.astype(float)
        anos_todo_intervalo = np.arange(0, 47)

        # Filtrar anos com dados de mortalidade não-zero
        anos_com_dados = anos[dados_mortalidade > 0]
        dados_mortalidade = dados_mortalidade[dados_mortalidade > 0]

        outlier_factor = 1.2

        IQR = iqr(dados_mortalidade)
        lower_bound = np.percentile(dados_mortalidade, 25) - outlier_factor * IQR
        upper_bound = np.percentile(dados_mortalidade, 75) + outlier_factor * IQR

        non_outliers_mask = (dados_mortalidade >= lower_bound) & (dados_mortalidade <= upper_bound)
        anos_com_dados = anos_com_dados[non_outliers_mask]
        dados_mortalidade = dados_mortalidade[non_outliers_mask]


        # Verificar se a tendência é de crescimento ou decrescimento
        coef_angular = np.polyfit(anos_com_dados, dados_mortalidade, 1)[0]
        if coef_angular >= 0:

            # Ajustar o modelo linear aos dados
            try:
                params, _ = curve_fit(modelo_linear, anos_com_dados, dados_mortalidade)
                constante, coeficiente = params
                previsao = modelo_linear(anos_todo_intervalo, constante, coeficiente)
                tipo_modelo = 'Linear'

            except RuntimeError:
                constante, coeficiente = np.nan, np.nan
                previsao = np.nan * anos
        else:
            # Ajustar o modelo exponencial aos dados
            try:
                params, _ = curve_fit(modelo_exponencial, anos_com_dados, dados_mortalidade, maxfev=10000)
                constante, coeficiente = params
                previsao = modelo_exponencial(anos_todo_intervalo, constante, coeficiente)
                tipo_modelo = 'Exponencial'

            except RuntimeError:
                constante, coeficiente = np.nan, np.nan
                previsao = np.nan * anos



        # Criar um DataFrame temporário com os resultados atuais
        df_temp = pd.DataFrame({'Município':[mun],'Sexo': [sexo], 'Faixa Etária': [faixa_etaria],
                                'Tipo de Modelo':[tipo_modelo],'Constante': [constante], 'Coeficiente': [coeficiente]})
        
        # Concatenar o DataFrame temporário ao DataFrame de resultados
        resultados = pd.concat([resultados, df_temp], ignore_index=True)

        
        # Plotar os dados originais e a curva ajustada
        row, col = divmod(i, num_colunas)
        axs[row, col].scatter(anos_com_dados + 2000, dados_mortalidade, label='Dados de mortalidade', marker='o')
        axs[row, col].plot(anos_todo_intervalo + 2000, previsao, label=f'Modelo {tipo_modelo} Ajustado', linestyle='--', color='red')
        axs[row, col].set_xlabel('Ano')
        axs[row, col].set_ylabel('Taxa de Óbitos')
        axs[row, col].set_title(f'Sexo: {sexo} | Faixa Etária: {faixa_etaria}')
        axs[row, col].grid(True)
        axs[row, col].legend()

    # Add a single legend below all the plots in the right corner
#    plt.legend(bbox_to_anchor=(.4, -.7), loc='lower left', borderaxespad=0.)


    fig.suptitle(f'Projeção de Mortalidade por Sexo e Faixa - {mun} \n', fontsize=15, fontweight='bold')
    plt.tight_layout()
#    plt.savefig(f'../../br_demography/results/fig/death_rate/teste/mortalidade_{mun}_non_outliers.svg', format="svg")
    plt.savefig(f'../../br_demography/results/fig/death_rate/norte_rmc/{mun}.png', dpi=300, bbox_inches='tight')
    plt.close(fig)




import warnings
warnings.filterwarnings('ignore')
warnings.filterwarnings("ignore", category=RuntimeWarning)


Restante fora Ribeira


In [74]:
new_columns = [*resultados.columns.tolist()]
new_columns.extend(range(2022, 2046))
resultados = resultados.reindex(columns=new_columns, fill_value=0)

In [75]:
for year in resultados.columns[6:].astype(int):
    resultados.loc[:,year] = year-2000

In [76]:
resultados_exp = resultados[resultados['Tipo de Modelo'] == 'Exponencial']
resultados_lin = resultados[resultados['Tipo de Modelo'] == 'Linear']

In [77]:
res = pd.concat(
    objs=[
        1 - modelo_exponencial(resultados_exp.iloc[:, 6:], resultados_exp.Constante.values.reshape((-1,1)), resultados_exp.Coeficiente.values.reshape((-1,1)))/1000
        , 1 - modelo_linear(resultados_lin.iloc[:, 6:], resultados_lin.Constante.values.reshape((-1,1)), resultados_lin.Coeficiente.values.reshape((-1,1)))/1000
        ]
    , axis=0
    
)


In [78]:
resultados = pd.concat(
    objs=[
        resultados_exp.iloc[:,:3]
        , resultados_lin.iloc[:,:3]
        ]
    , axis=0
    
)

In [79]:
resultados = resultados.join(res)

In [80]:
resultados

,Município,Sexo,Faixa Etária,2022,2023,2024,2025,2026,2027,2028,...,2036,2037,2038,2039,2040,2041,2042,2043,2044,2045
0,Norte e Sul,Feminino,0 a 9 anos,0.999598,0.999615,0.999630,0.999646,0.999660,0.999674,0.999687,...,0.999776,0.999785,0.999794,0.999802,0.999810,0.999818,0.999825,0.999833,0.999839,0.999846
1,Norte e Sul,Feminino,10 a 19 anos,0.999586,0.999590,0.999593,0.999596,0.999599,0.999602,0.999605,...,0.999628,0.999631,0.999634,0.999636,0.999639,0.999642,0.999645,0.999647,0.999650,0.999653
2,Norte e Sul,Feminino,20 a 29 anos,0.999352,0.999358,0.999365,0.999371,0.999378,0.999384,0.999390,...,0.999437,0.999443,0.999449,0.999454,0.999460,0.999465,0.999470,0.999476,0.999481,0.999486
3,Norte e Sul,Feminino,30 a 39 anos,0.998895,0.998908,0.998920,0.998933,0.998945,0.998957,0.998969,...,0.999061,0.999071,0.999082,0.999093,0.999103,0.999113,0.999124,0.999134,0.999144,0.999154
4,Norte e Sul,Feminino,40 a 49 anos,0.998017,0.998070,0.998121,0.998170,0.998219,0.998266,0.998312,...,0.998637,0.998673,0.998708,0.998742,0.998775,0.998808,0.998839,0.998870,0.998900,0.998929
5,Norte e Sul,Feminino,50 a 59 anos,0.994734,0.994855,0.994974,0.995090,0.995203,0.995314,0.995422,...,0.996202,0.996290,0.996375,0.996459,0.996541,0.996621,0.996699,0.996775,0.996849,0.996922
6,Norte e Sul,Feminino,60 a 69 anos,0.988396,0.988709,0.989013,0.989309,0.989596,0.989877,0.990149,...,0.992082,0.992295,0.992502,0.992704,0.992901,0.993092,0.993278,0.993459,0.993635,0.993807
7,Norte e Sul,Feminino,70 a 79 anos,0.970891,0.971620,0.972330,0.973023,0.973698,0.974357,0.974998,...,0.979587,0.980098,0.980597,0.981082,0.981556,0.982017,0.982468,0.982906,0.983334,0.983751
9,Norte e Sul,Masculino,0 a 9 anos,0.999604,0.999624,0.999642,0.999659,0.999676,0.999691,0.999706,...,0.999802,0.999812,0.999821,0.999830,0.999838,0.999846,0.999853,0.999860,0.999867,0.999873
12,Norte e Sul,Masculino,30 a 39 anos,0.996854,0.996877,0.996899,0.996922,0.996944,0.996966,0.996988,...,0.997158,0.997179,0.997199,0.997220,0.997240,0.997260,0.997280,0.997299,0.997319,0.997338


In [81]:
'''
resultados = pd.concat(
    objs=[
        resultados.iloc[:,:6]
        , modelo_exponencial(resultados.iloc[:, 4:], resultados.Constante.values.reshape((-1,1)), resultados.Coeficiente.values.reshape((-1,1)))
        ]
    , axis=1
    
)


resultados
'''

'\nresultados = pd.concat(\n    objs=[\n        resultados.iloc[:,:6]\n        , modelo_exponencial(resultados.iloc[:, 4:], resultados.Constante.values.reshape((-1,1)), resultados.Coeficiente.values.reshape((-1,1)))\n        ]\n    , axis=1\n    \n)\n\n\nresultados\n'

In [82]:
resultados.to_csv(path_or_buf='../../br_demography/results/tab/taxa_sobrevivencia_projetada_campina_2022_2044.csv', sep=';', decimal=',', encoding='utf-16')

In [83]:
resultados

,Município,Sexo,Faixa Etária,2022,2023,2024,2025,2026,2027,2028,...,2036,2037,2038,2039,2040,2041,2042,2043,2044,2045
0,Norte e Sul,Feminino,0 a 9 anos,0.999598,0.999615,0.999630,0.999646,0.999660,0.999674,0.999687,...,0.999776,0.999785,0.999794,0.999802,0.999810,0.999818,0.999825,0.999833,0.999839,0.999846
1,Norte e Sul,Feminino,10 a 19 anos,0.999586,0.999590,0.999593,0.999596,0.999599,0.999602,0.999605,...,0.999628,0.999631,0.999634,0.999636,0.999639,0.999642,0.999645,0.999647,0.999650,0.999653
2,Norte e Sul,Feminino,20 a 29 anos,0.999352,0.999358,0.999365,0.999371,0.999378,0.999384,0.999390,...,0.999437,0.999443,0.999449,0.999454,0.999460,0.999465,0.999470,0.999476,0.999481,0.999486
3,Norte e Sul,Feminino,30 a 39 anos,0.998895,0.998908,0.998920,0.998933,0.998945,0.998957,0.998969,...,0.999061,0.999071,0.999082,0.999093,0.999103,0.999113,0.999124,0.999134,0.999144,0.999154
4,Norte e Sul,Feminino,40 a 49 anos,0.998017,0.998070,0.998121,0.998170,0.998219,0.998266,0.998312,...,0.998637,0.998673,0.998708,0.998742,0.998775,0.998808,0.998839,0.998870,0.998900,0.998929
5,Norte e Sul,Feminino,50 a 59 anos,0.994734,0.994855,0.994974,0.995090,0.995203,0.995314,0.995422,...,0.996202,0.996290,0.996375,0.996459,0.996541,0.996621,0.996699,0.996775,0.996849,0.996922
6,Norte e Sul,Feminino,60 a 69 anos,0.988396,0.988709,0.989013,0.989309,0.989596,0.989877,0.990149,...,0.992082,0.992295,0.992502,0.992704,0.992901,0.993092,0.993278,0.993459,0.993635,0.993807
7,Norte e Sul,Feminino,70 a 79 anos,0.970891,0.971620,0.972330,0.973023,0.973698,0.974357,0.974998,...,0.979587,0.980098,0.980597,0.981082,0.981556,0.982017,0.982468,0.982906,0.983334,0.983751
9,Norte e Sul,Masculino,0 a 9 anos,0.999604,0.999624,0.999642,0.999659,0.999676,0.999691,0.999706,...,0.999802,0.999812,0.999821,0.999830,0.999838,0.999846,0.999853,0.999860,0.999867,0.999873
12,Norte e Sul,Masculino,30 a 39 anos,0.996854,0.996877,0.996899,0.996922,0.996944,0.996966,0.996988,...,0.997158,0.997179,0.997199,0.997220,0.997240,0.997260,0.997280,0.997299,0.997319,0.997338
